In [1]:
import json
import pandas as pd
import numpy as np
import ast
%load_ext autoreload
%autoreload 2
import Funciones
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime
import re

import warnings
warnings.filterwarnings("ignore")

In [2]:
ruta_reviews = r'..\Data\australian_user_reviews.json'

In [3]:
filas_items = []

with open(ruta_reviews, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        filas_items.append(ast.literal_eval(line))

In [4]:
df_reviews = pd.DataFrame(filas_items)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Se ve qué tipo de datos hay, y se revisa si hay valores nulos:

In [5]:
Funciones.verificar_tipo_datos(df_reviews)

,nombre,tipo_datos,porcentaje_no_nulos,porcentaje_nulos,nulos
0,user_id,[<class 'str'>],100.0,0.0,0
1,user_url,[<class 'str'>],100.0,0.0,0
2,reviews,[<class 'list'>],100.0,0.0,0


Habiendo verificado si hay nulos, se procede a verificar si hay elementos duplicados:

In [6]:

Funciones.duplicados_por_columna(df_reviews, 'user_id')


,user_id,user_url,reviews
12888,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
5250,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
3133,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
3134,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
4139,29123,http://steamcommunity.com/id/29123,"[{'funny': '', 'posted': 'Posted March 26.', '..."
...,...,...,...
2721,xXAussieRockXx,http://steamcommunity.com/id/xXAussieRockXx,"[{'funny': '', 'posted': 'Posted July 17, 2015..."
2680,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
5855,zeroblade,http://steamcommunity.com/id/zeroblade,"[{'funny': '', 'posted': 'Posted November 30, ..."


Se puede observar que hay 623 filas duplicadas. Se procede a la eliminación de duplicados.

In [7]:
# Se eliminan 
df_reviews = df_reviews.drop_duplicates('user_id', keep=False)



In [8]:
# Se vuelve a ver si sigue habiendo duplicados
Funciones.duplicados_por_columna(df_reviews, 'user_id')

'No hay duplicados'

Se observa la columna 'reviews':

In [9]:
df_reviews['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

Este archivo contiene las siguientes columnas:

•user_id: Es el código identificador único del usuario.
•user_url: Es la URL del perfil del usuario.
•reviews: Review de usuario en formato Json
    •funny: Aquí se ve si algún usuario puso algo gracioso.
    •posted: Fecha de posteo de la review.
    •last_edited: Fecha de la última edición.
    •item_id: Es el código identificador único del juego.
    •helpful: Aquí los demás usuarios aclaran si les fue útil la información.
    •recommend: Indica si el usuario recomienda o no recomienda el juego.
    •review: Aquí se ven los comentarios acerca del juego.

Se transforma y trata la columna 'reviews', que se encuentra añidada, se normaliza:

In [10]:
df_reviews_desañidado = pd.json_normalize(df_reviews['reviews'])
df_reviews_desañidado

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
25171,"{'funny': '', 'posted': 'Posted May 31.', 'las...",None,None,None,None,None,None,None,None,None
25172,"{'funny': '', 'posted': 'Posted June 17.', 'la...",None,None,None,None,None,None,None,None,None
25173,"{'funny': '1 person found this review funny', ...",None,None,None,None,None,None,None,None,None
25174,"{'funny': '', 'posted': 'Posted July 21.', 'la...","{'funny': '', 'posted': 'Posted July 10.', 'la...","{'funny': '', 'posted': 'Posted July 10.', 'la...","{'funny': '', 'posted': 'Posted July 8.', 'las...",None,None,None,None,None,None


Se procede a unir lo anterior con las columnas de 'user_id', y 'user_url':

In [11]:
df_reviews_desañidado = pd.concat([df_reviews[['user_id', 'user_url']], df_reviews_desañidado], axis=1)
df_reviews_desañidado

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,http://steamcommunity.com/id/doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,http://steamcommunity.com/id/maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
17819,NaN,NaN,"{'funny': '', 'posted': 'Posted April 11.', 'l...",None,None,None,None,None,None,None,None,None
17916,NaN,NaN,"{'funny': '', 'posted': 'Posted December 29, 2...","{'funny': '', 'posted': 'Posted June 25, 2015....",None,None,None,None,None,None,None,None
18028,NaN,NaN,"{'funny': '', 'posted': 'Posted June 23, 2014....",None,None,None,None,None,None,None,None,None
18234,NaN,NaN,"{'funny': '7 people found this review funny', ...","{'funny': '', 'posted': 'Posted June 13, 2014....","{'funny': '', 'posted': 'Posted May 14, 2014.'...","{'funny': '', 'posted': 'Posted June 5, 2014.'...","{'funny': '', 'posted': 'Posted June 13, 2014....","{'funny': '', 'posted': 'Posted June 17, 2015....","{'funny': '', 'posted': 'Posted May 1, 2015.',...","{'funny': '', 'posted': 'Posted March 26, 2015...","{'funny': '', 'posted': 'Posted August 16, 201...",None


In [12]:
Funciones.verificar_tipo_datos(df_reviews_desañidado)

,nombre,tipo_datos,porcentaje_no_nulos,porcentaje_nulos,nulos
0,user_id,"[<class 'str'>, <class 'float'>]",97.59,2.41,623
1,user_url,"[<class 'str'>, <class 'float'>]",97.59,2.41,623
2,0,"[<class 'dict'>, <class 'NoneType'>, <class 'f...",97.48,2.52,650
3,1,"[<class 'dict'>, <class 'NoneType'>, <class 'f...",45.56,54.44,14044
4,2,"[<class 'dict'>, <class 'NoneType'>, <class 'f...",27.82,72.18,18622
5,3,"[<class 'NoneType'>, <class 'dict'>, <class 'f...",18.14,81.86,21120
6,4,"[<class 'NoneType'>, <class 'dict'>, <class 'f...",12.44,87.56,22589
7,5,"[<class 'NoneType'>, <class 'dict'>, <class 'f...",8.73,91.27,23546
8,6,"[<class 'NoneType'>, <class 'dict'>, <class 'f...",5.92,94.08,24272
9,7,"[<class 'NoneType'>, <class 'dict'>, <class 'f...",3.83,96.17,24811


Usamos la función melt para transformar las columnas en filas.  La función melt() toma el DataFrame original y especifica las columnas 'user_id' y 'user_url' como identificadores únicos (id_vars). La columna 'reviews' contiene las reviews correspondientes. 

In [13]:
df_reviews_desañidado = pd.melt(df_reviews_desañidado, id_vars=['user_id', 'user_url'], 
                       value_vars=list(range(9)),
                       value_name='reviews')
df_reviews_desañidado

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,http://steamcommunity.com/id/js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,http://steamcommunity.com/id/doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,http://steamcommunity.com/id/maplemage,0,"{'funny': '3 people found this review funny', ..."
...,...,...,...,...
232186,NaN,NaN,8,None
232187,NaN,NaN,8,None
232188,NaN,NaN,8,None
232189,NaN,NaN,8,"{'funny': '', 'posted': 'Posted August 16, 201..."


Se eliminan los valores None que posee la columna 'reviews':

In [14]:
df_reviews_desañidado=df_reviews_desañidado.dropna()

Usamos la función json_normalize para expandir los diccionarios en la columna 'reviews' en varias columnas:

In [15]:
df_reviews = pd.json_normalize(df_reviews_desañidado['reviews']).add_prefix('reviews_')
df_reviews = pd.concat([df_reviews_desañidado[['user_id', 'user_url']], df_reviews], axis=1)
df_reviews

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud
...,...,...,...,...,...,...,...,...,...
55735,NaN,NaN,,"Posted December 20, 2013.","Last edited January 16, 2015.",105600,No ratings yet,True,","
55736,NaN,NaN,,"Posted February 3, 2014.",,221910,No ratings yet,True,"A first person game which has no guns, no line..."
55740,NaN,NaN,,"Posted August 2, 2014.",,304930,No ratings yet,True,Fun game with friends
55741,NaN,NaN,,"Posted July 31, 2015.",,265630,No ratings yet,True,So Fun!! :D


Se reemplazan los valores vacíos de la columna 'reviews_last_edited' como None:

In [16]:
df_reviews.replace('', None, inplace=True)
df_reviews

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,None,Posted February 3.,None,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,None,"Posted October 14, 2013.",None,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",None,211420,35 of 43 people (81%) found this review helpful,True,Git gud
...,...,...,...,...,...,...,...,...,...
55735,NaN,NaN,None,"Posted December 20, 2013.","Last edited January 16, 2015.",105600,No ratings yet,True,","
55736,NaN,NaN,None,"Posted February 3, 2014.",None,221910,No ratings yet,True,"A first person game which has no guns, no line..."
55740,NaN,NaN,None,"Posted August 2, 2014.",None,304930,No ratings yet,True,Fun game with friends
55741,NaN,NaN,None,"Posted July 31, 2015.",None,265630,No ratings yet,True,So Fun!! :D


In [17]:
Funciones.verificar_tipo_datos(df_reviews)

,nombre,tipo_datos,porcentaje_no_nulos,porcentaje_nulos,nulos
0,user_id,"[<class 'str'>, <class 'float'>]",75.19,24.81,18395
1,user_url,"[<class 'str'>, <class 'float'>]",75.19,24.81,18395
2,reviews_funny,"[<class 'NoneType'>, <class 'str'>, <class 'fl...",10.32,89.68,66485
3,reviews_posted,"[<class 'str'>, <class 'float'>]",75.19,24.81,18395
4,reviews_last_edited,"[<class 'NoneType'>, <class 'str'>, <class 'fl...",7.70,92.30,68431
5,reviews_item_id,"[<class 'str'>, <class 'float'>]",75.19,24.81,18395
6,reviews_helpful,"[<class 'str'>, <class 'float'>]",75.19,24.81,18395
7,reviews_recommend,"[<class 'bool'>, <class 'float'>]",75.19,24.81,18395
8,reviews_review,"[<class 'str'>, <class 'NoneType'>, <class 'fl...",75.15,24.85,18423


Se eliminan las columnas 'reviews_funny' y 'reviews_last_edited' por que al tener muchos valores faltantes, nos resulta engorroso y poco útil conservarlas:


In [18]:
df_reviews = df_reviews.drop(columns=['reviews_funny', 'reviews_last_edited'])
df_reviews.columns

Index(['user_id', 'user_url', 'reviews_posted', 'reviews_item_id',
       'reviews_helpful', 'reviews_recommend', 'reviews_review'],
      dtype='object')

Se transforma la columna 'reviews_posted', y se extrae el año:

In [19]:

df_reviews['reviews_posted'].fillna('sin fecha', inplace=True)

In [20]:
df_reviews['reviews_posted'] = df_reviews['reviews_posted'].apply(Funciones.extraer_anio)

In [21]:
df_reviews.head()

,user_id,user_url,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,2014,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,3,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,2013,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,2014,211420,35 of 43 people (81%) found this review helpful,True,Git gud


Se eliminan nulos:

In [22]:
#Se eliminan los nulos
df_reviews = df_reviews.dropna(subset=['reviews_review'])

In [23]:
df_reviews = df_reviews.dropna(subset=['user_id'])

In [24]:
df_reviews = df_reviews.dropna(subset=['user_url'])

Se guarda el dataframe final:

In [25]:
archivo_csv = r'..\Data\user_reviews.csv'
df_reviews.to_csv(archivo_csv, index=False, encoding='utf-8')

In [26]:
table = pa.Table.from_pandas(df_reviews)

In [27]:
nombre_archivo_parquet = 'users_reviews.parquet'

In [28]:
pq.write_table(table, nombre_archivo_parquet)